In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import log, log10
from torch.utils.data import Dataset
import data_processing as dp
import pickle
from privacy_policies_dataset import PrivacyPoliciesDataset
from os.path import isfile, join
from os import listdir
import numpy as np

In [ ]:
dictionary = dp.get_tokens("raw_data")

In [ ]:
len(dictionary)

In [ ]:
word2vector, word2idx_glove = dp.get_glove_dicts("glove.6B", 50)

In [ ]:
word2vector[None]

In [ ]:
word2idx_glove[None]

In [ ]:
weights_matrix, word2idx = dp.get_weight_matrix(dictionary, word2vector, 50)

In [ ]:
num_embeddings, embeddings_dim = weights_matrix.shape

In [ ]:
labels_file = open("labels.pkl","rb")

labels = pickle.load(labels_file)

labels_file.close()

In [ ]:
sentence_matrices, labels_matrices = dp.process_dataset(labels, word2idx)

In [ ]:
dp.get_aggregated_data(read=True)

In [ ]:
dataset = PrivacyPoliciesDataset("raw_data", word2idx, labels)

In [ ]:
dataset.resize_segments()

In [ ]:
segment_example, _ = dataset[0]

In [ ]:
myemb = nn.Embedding(num_embeddings, embeddings_dim)

myemb.from_pretrained(torch.tensor(weights_matrix))

In [ ]:
matrix = myemb(segment_example)

print("Before reshaping: " + str(matrix.shape))

# matrix = matrix.view(1,1,67,50)

# print("After reshaping: " + str(matrix.shape))

In [ ]:
conv = nn.Conv2d(1,2,(7, 50))

In [ ]:
output = conv(matrix)

In [ ]:
output.shape

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, weights_matrix):
        
        super(CNN, self).__init__()
        
        num_embeddings, embeddings_dim = weights_matrix.shape
        
        self.embedding = nn.Embedding(num_embeddings, embeddings_dim)
                
        self.first_channel = nn.Sequential(
            
            nn.Conv2d(1,2,(7, embeddings_dim)),
            
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2, stride=2)
            
        )
        
        self.second_channel = nn.Sequential(
            
            nn.Conv2d(1,2,(5, embeddings_dim)),
            
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2, stride=2)
            
        )
        
        self.third_channel = nn.Sequential(
            
            nn.Conv2d(1,2,(3, embeddings_dim)),
            
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2, stride=2)
            
        )
        
        self.deep_neural_network = nn.Sequential(
            
        )
    
    def forward(self):
        
        pass

Here we can see a very strange behaviour. We are expecting to have all 0s except for the last entry in which we are expecting Ln(0.9) and it is not even close to it. It seems they are not computing the BCE exactly as we think. 

In [ ]:
y_hat = torch.tensor([[1, 0, 0],[0, 1, 0.9]]).float()
y = torch.tensor([[1, 0, 0],[0, 1, 1]]).float()

In [ ]:
F.binary_cross_entropy(y,y_hat,reduction="none")

In [ ]:
loss=nn.BCELoss()
loss(y_hat,y)

### Things to take into consideration

1. It seems that with teh GloVe pretrained embeddings there are 1000 words that are missing and are initialized as random vectors.